link from : https://www.crisluengo.net/

# The Hessian and the Structure Tensor


In 2D, the Hessian of an image $I$ is defined as
$$
H=\nabla \nabla^{T} I=\left(\begin{array}{ll}
I_{x x} & I_{x y} \\
I_{x y} & I_{y y}
\end{array}\right)
$$
The Structure tensor is defined as
$$
S=\overline{(\nabla I)(\nabla I)^{T}}=\left(\begin{array}{cc}
\overline{I_{x}^{2}} & \overline{I_{x} I_{y}} \\
\overline{I_{x} I_{y}} & \overline{I_{y}^{2}}
\end{array}\right)
$$

Though the two written equatinos look fairly similar, the computations these equations represent are not that similar, and the properties of these tensor images are not either.

 However, we don’t compute it by applying two first-order partial derivatives in sequence. Instead, we **compute the second-order derivatives directly**. This is more efficient. Also, since this is a symmetric matrix, DIPlib computes and stores the off-diagonal component only once

# hessian matrix
The Hessian matrix is formed by the **second order partial derivatives**,
$$
\nabla \nabla^{T}=\left[\begin{array}{cccc}
\frac{\partial^{2}}{\partial x^{2}} & \frac{\partial^{2}}{\partial x \partial y} & \cdots & \frac{\partial^{2}}{\partial x \partial z} \\
\frac{\partial^{2}}{\partial x \partial y} & \frac{\partial^{2}}{\partial y^{2}} & \cdots & \frac{\partial^{2}}{\partial y \partial z} \\
\vdots & \vdots & \ddots & \vdots \\
\frac{\partial^{2}}{\partial x \partial z} & \frac{\partial^{2}}{\partial y \partial z} & \cdots & \frac{\partial^{2}}{\partial z^{2}}
\end{array}\right]
$$


In [42]:
import diplib as dip
import sys
import matplotlib.pyplot as plt

from skimage import util
def Cvt8bit(image):
    return util.img_as_ubyte(image)


sys.path.append('../')
import imgio
%matplotlib widget

img = imgio.ski_imread(r'..\Small_close-1.tif')
img = Cvt8bit(img)
#img=imgio.CvToGray(img)
H = dip.Hessian(img)
print(type(H))
plt.figure()
plt.imshow(img)
plt.figure()

dip.TileTensorElements(H).Show()
plt.figure()
dip.Trace(H).Show()
plt.figure()
dip.Determinant(H).Show()



<class 'diplib.PyDIP_bin.Image'>


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# structure tensor

The Structure Tensor is the outer product of the gradient vector with itself, locally averaged. It is thus formed exclusively by **first-order partial derivatives**


$$
S=\overline{(\nabla I)(\nabla I)^{T}}=\left[\begin{array}{cccc}
\overline{I_{x}^{2}} & \overline{I_{x} I_{y}} & \cdots & \overline{I_{x} I_{z}} \\
\overline{I_{x} I_{y}} & \overline{I_{y}^{2}} & \cdots & \overline{I_{y} I_{z}} \\
\vdots & \vdots & \ddots & \vdots \\
\overline{I_{x} I_{z}} & \overline{I_{y} I_{z}} & \cdots & \overline{I_{z}^{2}}
\end{array}\right]
$$


The gradient is always perpendicular to the line, but on one side it points one way, and on the other side it points in the opposite direction. These two vectors have **opposite direction**, but the same **orientation**.  

![direction and oriention](../images/hessian_vs_struct__line_and_gradients.svg)  

Local averaging would have these opposite vectors cancel out, whereas we would like to average their orientation, so as to determine the line’s orientation. A common trick计谋 around this is to **double the angles**, such that angles 180 degrees apart become the same. But then vectors 90 degrees apart will cancel out. So just a mapping of the angle to another angle will not allow us to preserve all information.   
Similarly, any mapping of 2D vectors to other 2D vectors will not allow us to preserve all information. We need to map the 2D vectors to a higher-dimensional space. The Structure Tensor maps the 2D vectors to a 3D space (there are four components, but two of those are identical).



In [38]:
g = dip.Gradient(img)
S = g * dip.Transpose(g)
#S = dip.Gauss(S, 3)
plt.figure()
dip.TileTensorElements(S).Show()
S = dip.StructureTensor(img, tensorSigmas=3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

To retrieve the information stored in the Structure Tensor, an eigenvalue analysis is typically applied. For each pixel we thus obtain two eigenvalues, and an eigenvector’s orientation (the other eigenvector is perpendicular to the first, and thus redundant).  

The **eigenvector encodes the orientation of the line**, the two **eigenvalues** encode the **gradient strength (energy)** and **variation (isotropy)**. Eigenvalue analysis **is typically represented as an ellipse**, with an orientation and two axes lengths corresponding to the eigenvalues.

![stick and ball](../images/stick_and_ball_ellipse.png)  

The larger the ellipse, the stronger the gradients in the neighborhood. The more elongated the ellipse, the more uniform are the edge orientations within the neighborhood. Conversely, the more circular the ellipse, the more varied are the edge orientations.

![corner stick plane](../images/corner.png)

If both eigenvalues are similar in magnitude, then the gradients in the local image region point in all directions, the neighborhood is **isotropic**.  

If one eigenvalue is much larger than the other, then the gradients in the local image region all point in the same direction (or rather, have the same orientation). 

There are two common ways to represent this  
**anisotropy measure** (with  the larger eigenvalue λ1 and  the smaller one λ2):

$$
\begin{aligned}
&\frac{\lambda_{1}-\lambda_{2}}{\lambda_{1}+\lambda_{2}} \\
\\
&1-\frac{\lambda_{2}}{\lambda_{1}}
\end{aligned}
$$

**energy measure**:
$${\lambda_{1}+\lambda_{2}}$$


> If we write this out, we notice that we’re just looking at the smallest eigenvalue:
$$
(1-\text { anisotropy } 1) \text { energy }=\left(1-\frac{\lambda_{1}-\lambda_{2}}{\lambda_{1}+\lambda_{2}}\right)\left(\lambda_{1}+\lambda_{2}\right)=\lambda_{1}+\lambda_{2}-\left(\lambda_{1}-\lambda_{2}\right)=2 \lambda_{2}
$$

>Because the eigenvalue analysis is rather expensive to compute, Harris and Stephens suggested an approximation computed using the determinant and the trace of the Structure Tensor,

please see **connerdetector.ipynb**



In [44]:
plt.figure()
(dip.Determinant(S) - 0.04 * dip.Trace(S)**2).Show()
plt.figure()
(dip.Determinant(S) / dip.Trace(S)).Show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
E, V = dip.EigenDecomposition(S)
plt.figure()
E(0).Show()
plt.figure()
E(1).Show()
dip.Angle(V(slice(0,1))).Show('orientation')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Conclusion
The Hessian and the Structure Tensor are both similar-looking mathematical expressions that have similar applications, but they have wildly differently computations behind them, and wildly different ways of extracting relevant information. We’ve seen how both of them can be used to detect key points (corners, features to track, etc.).  
They both can also be used to **detect lines**(for example the very popular Frangi vesselness measure is based on the Hessian) and many other things. The **Structure Tensor** is also used in **Lucas-Kanade optical flow**, and in **Weickert’s coherence enhancing diffusion**.